## Imports

In [ ]:
import os
import sys
from pathlib import Path

def _find_repo_root(start: Path) -> Path:
    """Find project root even when notebook is launched from a subfolder."""
    candidates = [start, *start.parents]

    for candidate in candidates:
        if (
            (candidate / "code" / "utils").exists()
            and (candidate / "sourcedata").exists()
            and (candidate / "derivatives").exists()
        ):
            return candidate

    for candidate in candidates:
        if (candidate / "code" / "utils").exists():
            return candidate

    return start


repo_root = _find_repo_root(Path.cwd().resolve())
code_root = repo_root / "code"

for p in (repo_root, code_root):
    p_str = str(p)
    if p.exists() and p_str not in sys.path:
        sys.path.insert(0, p_str)

project_code_root = code_root if (code_root / "utils").exists() else repo_root
if str(project_code_root) not in sys.path:
    sys.path.insert(0, str(project_code_root))

print(f"Resolved repo root: {repo_root}")

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from utils import autocorrelation as acf
from utils import helper_functions as hf
from IPython import get_ipython


---
### Load subjects 

In [ ]:
subjects = ["secundo", "gus"]

# repo_root = Path.cwd().resolve()
# if not (repo_root / "sourcedata").exists() and (repo_root.parent / "sourcedata").exists():
#     repo_root = repo_root.parent
source_root = repo_root / "sourcedata"
deriv_root = repo_root / "derivatives" / "preprocessing"

ANALYSIS_MODES = ["raw", "mean_divide", "zscore"]
DATA_MODE = "zscore"

EDA_ROOT_NAME = "eda"
ACF_QC_FILENAME = "acf_qc_summary.csv"
FDIFF_ROBUST_PCTL = (2.0, 98.0)
SPATIAL_EPS = 1e-8
SPATIAL_MIN_VALID_SAMPLES = 8


def _ensure_acf_configured():
    acf.configure(
        deriv_root_path=deriv_root,
        eda_root_name=EDA_ROOT_NAME,
        acf_qc_filename=ACF_QC_FILENAME,
        frame_diff_robust_pctl=FDIFF_ROBUST_PCTL,
        spatial_eps=SPATIAL_EPS,
        spatial_min_valid_samples=SPATIAL_MIN_VALID_SAMPLES,
    )


_ensure_acf_configured()


---
### Load Baseline Data

Load baseline sessions for model training.

In [ ]:
for subject in subjects:
    data_directory = source_root / subject
    data_output_dir = deriv_root / subject  # subject-specific output folder
    data_output_dir.mkdir(parents=True, exist_ok=True)
    if DATA_MODE == "raw":
        baseline_output_dir = os.path.join(data_output_dir, "baseline_only")
    else:
        baseline_output_dir = os.path.join(
            data_output_dir, f"baseline_only_normalized/{DATA_MODE}"
        )
    # Load all baseline sessions
    baseline_sessions = hf.load_saved_baseline_sessions(baseline_output_dir)

    # Print summary
    if len(baseline_sessions) > 0:
        print(f"\nBaseline Data Summary:{subject} ({DATA_MODE})")
        print(f"  Total sessions: {len(baseline_sessions)}")
        total_frames = sum(s["frames"].shape[0] for s in baseline_sessions)
        print(f"  Total baseline frames: {total_frames:,}")

        spatial_shapes = [s["frames"].shape[1:] for s in baseline_sessions]
        unique_shapes = set(spatial_shapes)
        print(f"  Spatial dimensions: {unique_shapes}")

        # Show frame count distribution
        frame_counts = [s["frames"].shape[0] for s in baseline_sessions]
        print(
            f"  Frames per session: min={min(frame_counts)}, max={max(frame_counts)}, "
            f"mean={np.mean(frame_counts):.0f}, std={np.std(frame_counts):.0f}"
        )

        # first session
        if len(baseline_sessions) > 0:
            first_session = baseline_sessions[0]
            print(f"\n  First session ({first_session['session_id']}):")
            print(f"    Frames: {first_session['frames'].shape[0]}")
            print(f"    Shape: {first_session['frames'].shape}")
            print(f"    Dtype: {first_session['frames'].dtype}")
            print(
                f"Value range: [{first_session['frames'].min():.2f}, {first_session['frames'].max():.2f}]"
            )

---
## Exploratory data analysis

---

### 2. Distributions
will display just a sample acquisition, but will save plots for each acquisition
#### Histograms of intensities
per acquisiton 

In [ ]:
USE_WIDGET = False
ipy = get_ipython()
backend_mode = "inline"

if ipy is not None:
    try:
        ipy.run_line_magic("matplotlib", "inline")
    except Exception:
        pass

print(f"Matplotlib backend: {matplotlib.get_backend()} | mode={backend_mode}")

subjects = ["secundo", "gus"]
sample_subject = subjects[0]  # change if needed
sample_session_idx = 2  # change if needed
sample_frame_idx = 0  # change if needed

baseline_output_dir = deriv_root / sample_subject / "baseline_only"
baseline_sessions = hf.load_all_baseline(str(baseline_output_dir))

In [ ]:
DIST_N_BINS = 120
DIST_MAX_POINTS = 500_000  # random subsample for speed in histogram ops
DIST_MODE_ORDER = list(ANALYSIS_MODES)
DIST_MODE_RANGES = {
    # "raw": None,
    "mean_divide": (-0.25, 0.25),
    "zscore": (-3.0, 3.0),
}

EDA_ROOT_NAME = "eda"

In [ ]:
def _eda_subject_root(subject):
    root = deriv_root / subject / EDA_ROOT_NAME / "distribution"
    root.mkdir(parents=True, exist_ok=True)
    return root


def _mode_transform(frames, mode, subject, session_id):
    arr = hf.load_saved_session_frames(deriv_root, subject, session_id, mode)
    if arr.shape != frames.shape:
        raise ValueError(
            f"Shape mismatch for {subject}/{session_id}/{mode}: raw={frames.shape}, saved={arr.shape}"
        )
    return arr


def _flatten_for_hist(arr, max_points=DIST_MAX_POINTS, seed=42):
    v = arr.reshape(-1)
    n = v.size
    if n <= max_points:
        return v
    rng = np.random.default_rng(seed)
    idx = rng.choice(n, size=max_points, replace=False)
    return v[idx]


def _subject_raw_range(baseline_sessions, q_low=1.0, q_high=99.0):
    lows, highs = [], []
    for sess in baseline_sessions:
        f = sess["frames"].astype(np.float32, copy=False)
        lows.append(float(np.percentile(f, q_low)))
        highs.append(float(np.percentile(f, q_high)))
    lo = float(np.median(lows)) if len(lows) else 0.0
    hi = float(np.median(highs)) if len(highs) else 1.0
    if hi <= lo:
        hi = lo + 1e-6
    return lo, hi


for subject in subjects:
    subject_root = _eda_subject_root(subject)
    out_dir = subject_root / "hist_per_acquisition"
    out_dir.mkdir(parents=True, exist_ok=True)

    baseline_output_dir = deriv_root / subject / "baseline_only"
    baseline_sessions = hf.load_all_baseline(str(baseline_output_dir))

    if len(baseline_sessions) == 0:
        print(
            f"No baseline sessions for {subject}; skipping per-acquisition histograms"
        )
        continue

    # subject-specific robust range for raw
    raw_lo, raw_hi = _subject_raw_range(baseline_sessions)
    mode_ranges = dict(DIST_MODE_RANGES)
    mode_ranges["raw"] = (raw_lo, raw_hi)

    print(
        f"\n[{subject}] per-acquisition histograms | sessions={len(baseline_sessions)}"
    )

    for s_idx, sess in enumerate(baseline_sessions):
        session_id = sess["session_id"]
        raw = sess["frames"].astype(np.float32, copy=False)

        fig, axes = plt.subplots(1, 3, figsize=(15, 4.2))

        for ax, mode in zip(axes, DIST_MODE_ORDER, strict=False):
            arr = _mode_transform(raw, mode, subject, session_id)
            vals = _flatten_for_hist(arr, max_points=DIST_MAX_POINTS, seed=42 + s_idx)

            lo, hi = mode_ranges[mode]
            vals_plot = np.clip(vals, lo, hi)

            ax.hist(
                vals_plot,
                bins=DIST_N_BINS,
                range=(lo, hi),
                density=True,
                alpha=0.9,
                color="steelblue",
            )
            
            ax.set_title(mode)
            ax.set_xlabel("value")
            ax.set_ylabel("density")
            ax.grid(alpha=0.2)

        fig.suptitle(f"{subject} | session {session_id} | intensity histograms")
        fig.tight_layout()

        out_path = out_dir / f"{subject}_{session_id}_hist_triplet.png"
        fig.savefig(out_path, dpi=180)

        if subject == sample_subject and s_idx == sample_session_idx:
            plt.show()
        else:
            plt.close(fig)

    print(f"Saved per-acquisition histograms to: {out_dir}")

#### mean/median/max intensity values over time
per acquisition

In [ ]:
def compute_framewise_stats(frames_3d):
    arr = np.asarray(frames_3d, dtype=np.float32)
    flat = arr.reshape(arr.shape[0], -1)

    p25 = np.nanpercentile(flat, 25.0, axis=1)
    p75 = np.nanpercentile(flat, 75.0, axis=1)

    dvars = np.full(flat.shape[0], np.nan, dtype=np.float32)
    if flat.shape[0] > 1:
        dvars[1:] = np.nanmean(np.abs(flat[1:] - flat[:-1]), axis=1)

    return {
        "mean": np.nanmean(flat, axis=1),
        "median": np.nanmedian(flat, axis=1),
        # "p99": np.nanpercentile(flat, 99.0, axis=1),
        # "p99_9": np.nanpercentile(flat, 99.9, axis=1),
        "std": np.nanstd(flat, axis=1),
        "iqr": p75 - p25,
        "dvars": dvars,
    }


TARGET_MODES = list(ANALYSIS_MODES)

for subject in subjects:
    subject_root = _eda_subject_root(subject)
    out_dir = subject_root / "time_stats_per_acquisition"
    out_dir.mkdir(parents=True, exist_ok=True)

    baseline_output_dir = deriv_root / subject / "baseline_only"
    baseline_sessions = hf.load_all_baseline(str(baseline_output_dir))

    if len(baseline_sessions) == 0:
        print(
            f"No baseline sessions for {subject}; skipping per-acquisition time stats"
        )
        continue
    print(
        f"\n[{subject}] per-acquisition robust framewise stats | sessions={len(baseline_sessions)}"
    )

    for s_idx, sess in enumerate(baseline_sessions):
        session_id = acf.canonical_session_id(sess["session_id"], TARGET_MODES)

        stats = {}
        frame_count = None
        missing_modes = []

        for mode in TARGET_MODES:
            try:
                arr = hf.load_saved_session_frames(
                    deriv_root,
                    subject,
                    session_id,
                    mode,
                )
                arr = acf.squeeze_frames(arr).astype(np.float32, copy=False)
            except Exception as e:
                print(
                    f"{subject} | session {session_id} | mode {mode}: failed to load ({e})"
                )
                missing_modes.append(mode)
                continue

            if frame_count is None:
                frame_count = arr.shape[0]
            elif arr.shape[0] != frame_count:
                print(
                    f"{subject} | session {session_id}: frame count mismatch "
                    f"({frame_count} vs {arr.shape[0]}) across modes; skipping session"
                )
                missing_modes = TARGET_MODES.copy()
                break

            stats[mode] = compute_framewise_stats(arr)

        if missing_modes:
            print(
                f"{subject} | session {session_id}: missing/invalid modes {missing_modes}; skipping"
            )
            continue

        t = np.arange(frame_count)
        colors = {"raw": "tab:green", "mean_divide": "tab:orange", "zscore": "tab:blue"}
        linestyles = {
            "mean": "-",
            "median": "--",
            # "p99": "-",
            # "p99_9": "--",
            "std": "-",
            "iqr": "--",
            "dvars": "-",
        }

        panel_defs = [
            ("mean + median", ["mean", "median"]),
            # ("p99 + p99.9", ["p99", "p99_9"]),
            ("std + iqr", ["std", "iqr"]),
            ("dvars (frame 0 = NaN)", ["dvars"]),
        ]

        fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
        for ax, (title, metrics) in zip(axes, panel_defs, strict=False):
            for mode in TARGET_MODES:
                for metric in metrics:
                    ax.plot(
                        t,
                        stats[mode][metric],
                        color=colors.get(mode, None),
                        linestyle=linestyles[metric],
                        linewidth=1.4,
                        label=f"{mode} {metric}",
                    )
            ax.set_title(title)
            ax.set_ylabel("value")
            ax.grid(alpha=0.2)
            ax.legend(fontsize=8, ncol=2, loc="best")

        axes[-1].set_xlabel("Frame index")
        fig.suptitle(f"{subject} | session {session_id} | framewise stats")
        fig.tight_layout(rect=[0, 0, 1, 0.97])

        out_fig = out_dir / f"{subject}_{session_id}_time_stats_triplet.png"
        fig.savefig(out_fig, dpi=180)

        rows = []
        for mode in TARGET_MODES:
            for i in range(len(t)):
                rows.append(
                    {
                        "frame": int(i),
                        "mode": mode,
                        "mean": float(stats[mode]["mean"][i]),
                        "median": float(stats[mode]["median"][i]),
                        # "p99": float(stats[mode]["p99"][i]),
                        # "p99_9": float(stats[mode]["p99_9"][i]),
                        "std": float(stats[mode]["std"][i]),
                        "iqr": float(stats[mode]["iqr"][i]),
                        "dvars": float(stats[mode]["dvars"][i])
                        if np.isfinite(stats[mode]["dvars"][i])
                        else np.nan,
                    }
                )
        pd.DataFrame(rows).to_csv(
            out_dir / f"{subject}_{session_id}_time_stats_triplet.csv", index=False
        )

        if subject == sample_subject and s_idx == sample_session_idx:
            plt.show()
        else:
            plt.close(fig)

    print(f"Saved per-acquisition time stats to: {out_dir}")

### 4. Frame to frame difference
per acquisition

In [ ]:
# setup
if "ANALYSIS_MODES" not in globals():
    ANALYSIS_MODES = ["raw", "mean_divide", "zscore"]
FRAME_DIFF_METHOD = "consecutive"
ACF_MAX_LAG = 500
PSD_NPERSEG = 256
DEFAULT_FS_HZ = 1.0
FDIFF_ROBUST_PCTL = (2.0, 98.0)
FDIFF_ROOT_SUBDIR = "frame_diff"
AUTOCORR_ROOT_SUBDIR = "autocorr_psd"
SPATIAL_ROOT_SUBDIR = "spatial_autocorr"
SPATIAL_NEIGHBORHOODS = (4, 8)
SPATIAL_EPS = 1e-8
SPATIAL_MIN_VALID_SAMPLES = 8
CV_MODE_ORDER = list(ANALYSIS_MODES)
PIX_MODE_ORDER = list(ANALYSIS_MODES)
PIX_EPS = 1e-8
PIX_ROBUST_PCTL = (1.0, 99.0)
PIX_CV_CLIP = 5.0
PIX_ROOT_SUBDIR = "pixel_stability"

# Global-signal analyses deliberately exclude zscore.
# zscore spatial means cancel/reweight signal and are not directly comparable.
GLOBAL_SIGNAL_MODES = [m for m in ["raw", "mean_divide"] if m in ANALYSIS_MODES]
if len(GLOBAL_SIGNAL_MODES) == 0 and len(ANALYSIS_MODES) > 0:
    GLOBAL_SIGNAL_MODES = [ANALYSIS_MODES[0]]
print(f"Global-signal modes for ACF/PSD: {GLOBAL_SIGNAL_MODES}")

# Spatial alignment is estimated from a raw-derived reference, then reused for all modes.
SPATIAL_ALIGN_REF_MODE = "raw"
SPATIAL_ALIGN_REF_STAT = "mean"  # "mean" or "std"
SPATIAL_ALIGN_MAX_SHIFT_FRAC = 0.20
SPATIAL_ALIGN_MIN_SCORE = 0.20


ACF_REQUIRE_QC = True
ACF_QC_FILENAME = "acf_qc_summary.csv"
SPATIAL_REQUIRE_QC = ACF_REQUIRE_QC

acf.configure(
    deriv_root_path=deriv_root,
    eda_root_name=EDA_ROOT_NAME,
    acf_qc_filename=ACF_QC_FILENAME,
    frame_diff_robust_pctl=FDIFF_ROBUST_PCTL,
    spatial_eps=SPATIAL_EPS,
    spatial_min_valid_samples=SPATIAL_MIN_VALID_SAMPLES,
)

# Backward-compatible aliases for older notebook cells
_load_acf_qc_map_for_subject = acf.load_acf_qc_map_for_subject
_acf_qc_allows_session = acf.acf_qc_allows_session
_session_global_signal = acf.session_global_signal


In [ ]:
if "_ensure_acf_configured" in globals():
    _ensure_acf_configured()
else:
    acf.configure(deriv_root_path=deriv_root)

for subject in subjects:
    baseline_dir = deriv_root / subject / "baseline_only"
    baseline_sessions = hf.load_all_baseline(str(baseline_dir))
    print(
        f"\\n[{subject}] frame-to-frame diff per acquisition | sessions={len(baseline_sessions)}"
    )
    if len(baseline_sessions) == 0:
        continue

    out_dir = acf.analysis_subject_root(subject, FDIFF_ROOT_SUBDIR) / "per_session"
    out_dir.mkdir(parents=True, exist_ok=True)

    for s_idx, session in enumerate(baseline_sessions):
        session_id = session["session_id"]

        for mode in ANALYSIS_MODES:
            try:
                arr, metadata, _ = acf.load_saved_session_bundle(
                    deriv_root, subject, session_id, mode
                )
            except Exception as e:
                print(f"  - skip {session_id} | {mode}: {e}")
                pd.DataFrame(
                    [
                        {
                            "subject": subject,
                            "session_id": session_id,
                            "mode": mode,
                            "status": "missing_or_invalid",
                            "reason": str(e),
                        }
                    ]
                ).to_csv(
                    out_dir / f"{subject}_{session_id}_{mode}_frame_diff_summary.csv",
                    index=False,
                )
                continue

            if arr.shape[0] < 2:
                msg = f"not enough frames ({arr.shape[0]})"
                print(f"  - skip {session_id} | {mode}: {msg}")
                pd.DataFrame(
                    [
                        {
                            "subject": subject,
                            "session_id": session_id,
                            "mode": mode,
                            "status": "insufficient_frames",
                            "reason": msg,
                        }
                    ]
                ).to_csv(
                    out_dir / f"{subject}_{session_id}_{mode}_frame_diff_summary.csv",
                    index=False,
                )
                continue

            if FRAME_DIFF_METHOD != "consecutive":
                raise ValueError(
                    f"Unsupported FRAME_DIFF_METHOD={FRAME_DIFF_METHOD!r}; expected 'consecutive'"
                )

            diff = arr[1:] - arr[:-1]
            context_mean = np.nanmean(arr, axis=0)
            diff_mean_map = np.nanmean(diff, axis=0)
            diff_std_map = np.nanstd(diff, axis=0)
            diff_abs_mean_map = np.nanmean(np.abs(diff), axis=0)

            context_vmin, context_vmax = acf.robust_limits(
                context_mean, pctl=FDIFF_ROBUST_PCTL
            )
            dm_vmin, dm_vmax = acf.robust_limits(
                diff_mean_map, pctl=FDIFF_ROBUST_PCTL, symmetric=True
            )
            ds_vmin, ds_vmax = acf.robust_limits(
                diff_std_map, pctl=FDIFF_ROBUST_PCTL, nonnegative=True
            )
            da_vmin, da_vmax = acf.robust_limits(
                diff_abs_mean_map, pctl=FDIFF_ROBUST_PCTL, nonnegative=True
            )

            fig, axes = plt.subplots(2, 2, figsize=(11, 9))
            panels = [
                (
                    context_mean,
                    "viridis",
                    (context_vmin, context_vmax),
                    "mean frame (context)",
                ),
                (diff_mean_map, "seismic", (dm_vmin, dm_vmax), "mean frame diff"),
                (diff_std_map, "viridis", (ds_vmin, ds_vmax), "std frame diff"),
                (diff_abs_mean_map, "magma", (da_vmin, da_vmax), "mean abs frame diff"),
            ]
            for ax, (img, cmap, (vmin, vmax), title) in zip(axes.ravel(), panels, strict=False):
                im = ax.imshow(img, cmap=cmap, vmin=vmin, vmax=vmax)
                ax.set_title(f"{subject} | {session_id} | {mode} | {title}")
                ax.set_xlabel("x")
                ax.set_ylabel("y")
                fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

            fig.tight_layout()
            out_fig = out_dir / f"{subject}_{session_id}_{mode}_frame_diff_maps.png"
            fig.savefig(out_fig, dpi=180)

            if subject == sample_subject and s_idx == sample_session_idx:
                plt.show()
            else:
                plt.close(fig)

            n_finite = int(np.isfinite(diff).sum())
            fps = np.nan
            if "source_fps" in metadata:
                try:
                    fps = float(np.asarray(metadata["source_fps"]).item())
                except Exception:
                    fps = np.nan

            summary = {
                "subject": subject,
                "session_id": session_id,
                "mode": mode,
                "status": "ok",
                "n_frames": int(arr.shape[0]),
                "n_diff_frames": int(diff.shape[0]),
                "height": int(arr.shape[1]),
                "width": int(arr.shape[2]),
                "n_finite_diff_values": n_finite,
                "global_diff_mean": float(np.nanmean(diff)),
                "global_diff_std": float(np.nanstd(diff)),
                "global_diff_abs_mean": float(np.nanmean(np.abs(diff))),
                "source_fps": fps,
            }
            pd.DataFrame([summary]).to_csv(
                out_dir / f"{subject}_{session_id}_{mode}_frame_diff_summary.csv",
                index=False,
            )

    print(f"Saved per-acquisition frame-diff outputs to: {out_dir}")

across acquisitions

In [ ]:
if "_ensure_acf_configured" in globals():
    _ensure_acf_configured()
else:
    acf.configure(deriv_root_path=deriv_root)

for subject in subjects:
    baseline_dir = deriv_root / subject / "baseline_only"
    baseline_sessions = hf.load_all_baseline(str(baseline_dir))
    print(
        f"\\n[{subject}] frame-to-frame diff across acquisitions | sessions={len(baseline_sessions)}"
    )
    if len(baseline_sessions) == 0:
        continue

    out_dir = acf.analysis_subject_root(subject, FDIFF_ROOT_SUBDIR) / "across_sessions"
    out_dir.mkdir(parents=True, exist_ok=True)

    for mode in ANALYSIS_MODES:
        session_maps_abs = []
        session_maps_std = []
        used_session_ids = []
        skipped_shape = 0
        skipped_other = 0
        ref_shape = None

        for session in baseline_sessions:
            session_id = session["session_id"]
            try:
                arr, _, _ = acf.load_saved_session_bundle(
                    deriv_root, subject, session_id, mode
                )
                if arr.shape[0] < 2:
                    skipped_other += 1
                    continue

                if FRAME_DIFF_METHOD != "consecutive":
                    raise ValueError(
                        f"Unsupported FRAME_DIFF_METHOD={FRAME_DIFF_METHOD!r}"
                    )
                diff = arr[1:] - arr[:-1]

                shape_hw = arr.shape[1:]
                if ref_shape is None:
                    ref_shape = shape_hw
                elif shape_hw != ref_shape:
                    skipped_shape += 1
                    continue

                session_maps_abs.append(np.nanmean(np.abs(diff), axis=0))
                session_maps_std.append(np.nanstd(diff, axis=0))
                used_session_ids.append(session_id)
            except Exception:
                skipped_other += 1

        total_sessions = len(baseline_sessions)
        used_sessions = len(used_session_ids)

        if used_sessions == 0:
            pd.DataFrame(
                [
                    {
                        "subject": subject,
                        "mode": mode,
                        "status": "no_valid_sessions",
                        "n_sessions_total": int(total_sessions),
                        "n_sessions_used": 0,
                        "n_sessions_skipped_shape": int(skipped_shape),
                        "n_sessions_skipped_other": int(skipped_other),
                        "shape_ref": "",
                    }
                ]
            ).to_csv(
                out_dir / f"{subject}_{mode}_frame_diff_across_summary.csv", index=False
            )
            print(f"  - skip mode={mode}: no valid sessions")
            continue

        ABS = np.stack(session_maps_abs, axis=0)
        STD = np.stack(session_maps_std, axis=0)

        abs_across_mean = np.nanmean(ABS, axis=0)
        abs_across_std = np.nanstd(ABS, axis=0)
        std_across_mean = np.nanmean(STD, axis=0)
        std_across_std = np.nanstd(STD, axis=0)

        fig, axes = plt.subplots(2, 2, figsize=(11, 9))
        panels = [
            (
                abs_across_mean,
                "magma",
                acf.robust_limits(
                    abs_across_mean, pctl=FDIFF_ROBUST_PCTL, nonnegative=True
                ),
                f"{subject} | {mode} | across-acq mean(abs diff)",
            ),
            (
                abs_across_std,
                "viridis",
                acf.robust_limits(
                    abs_across_std, pctl=FDIFF_ROBUST_PCTL, nonnegative=True
                ),
                f"{subject} | {mode} | across-acq std(abs diff)",
            ),
            (
                std_across_mean,
                "viridis",
                acf.robust_limits(
                    std_across_mean, pctl=FDIFF_ROBUST_PCTL, nonnegative=True
                ),
                f"{subject} | {mode} | across-acq mean(diff std)",
            ),
            (
                std_across_std,
                "viridis",
                acf.robust_limits(
                    std_across_std, pctl=FDIFF_ROBUST_PCTL, nonnegative=True
                ),
                f"{subject} | {mode} | across-acq std(diff std)",
            ),
        ]

        for ax, (img, cmap, (vmin, vmax), title) in zip(axes.ravel(), panels):
            im = ax.imshow(img, cmap=cmap, vmin=vmin, vmax=vmax)
            ax.set_title(title)
            ax.set_xlabel("x")
            ax.set_ylabel("y")
            fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

        fig.tight_layout()
        out_fig = out_dir / f"{subject}_{mode}_frame_diff_across_maps.png"
        fig.savefig(out_fig, dpi=180)

        if subject == sample_subject:
            plt.show()
        else:
            plt.close(fig)

        summary_row = {
            "subject": subject,
            "mode": mode,
            "status": "ok",
            "n_sessions_total": int(total_sessions),
            "n_sessions_used": int(used_sessions),
            "n_sessions_skipped_shape": int(skipped_shape),
            "n_sessions_skipped_other": int(skipped_other),
            "shape_ref": str(tuple(int(x) for x in ref_shape)),
            "used_session_ids": "|".join(str(s) for s in used_session_ids),
            "global_abs_diff_mean": float(np.nanmean(abs_across_mean)),
            "global_abs_diff_std": float(np.nanstd(abs_across_mean)),
            "global_diff_std_mean": float(np.nanmean(std_across_mean)),
            "global_diff_std_std": float(np.nanstd(std_across_mean)),
        }
        pd.DataFrame([summary_row]).to_csv(
            out_dir / f"{subject}_{mode}_frame_diff_across_summary.csv", index=False
        )

    print(f"Saved across-acquisition frame-diff outputs to: {out_dir}")